In [ ]:
# ✅ Install dependencies
!pip install -U langchain langchain-community faiss-cpu sentence-transformers transformers gradio pypdf

In [11]:
# ✅ Load and split PDF into chunks
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
def load_pdf_chunks(pdf_path):
  doc_text = PyPDFLoader(pdf_path).load()
  print(doc_text)
  splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  chunks = splitter.split_documents(doc_text)
  return chunks

In [5]:
# ✅ Create FAISS vector DB using sentence transformer
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
def create_vector_db(chunks):
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = FAISS.from_documents(chunks, embeddings)
  return vector_db

In [6]:
# ✅ Generate answer using Flan-T5
from transformers import pipeline
def generate_answer(query,pdf_path):
  chunks = load_pdf_chunks(pdf_path)
  vector_db = create_vector_db(chunks)
  similar_docs = vector_db.similarity_search(query,k=3)
  context = "\n".join([doc.page_content for doc in similar_docs])
  flan = pipeline(task = 'text2text-generation', model = 'google/flan-t5-base')
  prompt = f"Answer the question based on the context:\n\n{context}\n\nQuestion: {query}\nAnswer:"
  answer = flan(prompt)[0]['generated_text']
  return answer

In [13]:
gr.Interface(
    fn=chat_with_pdf,
    inputs=[gr.File(label="Upload PDF"), gr.Textbox(label="Ask a question")],
    outputs="text",
    title="📄 Chat with your PDF",
    description="Uses Flan-T5 + LangChain to answer from your PDF"
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b86e17c59fbd3a5a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
